# day 12

https://adventofcode.com/2019/day/12

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day12.txt')

LOGGER = logging.getLogger('day12')

## part 1

### problem statement:

#### loading data

In [ ]:
logging.getLogger('matplotlib').setLevel(logging.ERROR)

import re

import numpy as np
import pandas as pd

In [ ]:
def parse_input_str(s):
    return pd.DataFrame(
        np.array([[float(elem)
                   for elem in re.match('<x=([\-\d]+), y=([\-\d]+), z=([\-\d]+)>', row).groups()]
                  for row in s.split('\n')]),
        columns=['x', 'y', 'z'])

In [ ]:
test_0 = """<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>"""

test_1 = """<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>"""

In [ ]:
s = """<x=10, y=15, z=7>
<x=15, y=10, z=0>
<x=20, y=12, z=3>
<x=0, y=-3, z=13>"""

def load_data():
    return s

In [ ]:
print(load_data())

#### function def

In [ ]:
def q_1(position_str, n_steps=1000, log_freq=100):
    position = parse_input_str(position_str)
    velocity = pd.DataFrame(np.zeros(position.shape),
                            columns=position.columns)
    
    N_MOONS = position.shape[0]
    for time_step in range(n_steps):
        #if time_step % log_freq == 0:
        #    LOGGER.debug(f'position = {position}')
        #    LOGGER.debug(f'velocity = {velocity}')
            
        # calculate gravity and update velocty
        for i in range(N_MOONS - 1):
            g_i = position.iloc[i + 1:] - position.iloc[i]
            g_i = (g_i / g_i.abs()).fillna(0)
            velocity.iloc[i] += g_i.sum(axis=0)
            velocity.iloc[i + 1:] -= g_i
        
        # update positions
        position += velocity
    
    # calculate energy
    pot = position.abs().sum(axis=1)
    kin = velocity.abs().sum(axis=1)
    total_energy = (pot * kin).sum()
    #LOGGER.debug(f'pot = {pot}')
    #LOGGER.debug(f'kin = {kin}')
    return total_energy

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_0, 10, 1) == 179
    assert q_1(test_1, 100, 10) == 1940
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data(), 1000)

## part 2

### problem statement:

#### function def

dropping pandas stuff (way too slow) and refactoring a bit of an answer posted to the subreddit

In [ ]:
import math

MP = parse_input_str(load_data()).to_dict(orient='records')
MV = [{c: 0 for c in 'xyz'} for row in MP]

In [ ]:
def tick():
    for i in range(len(MP)):
        for j in range(len(MP)):
            for k in MP[i]:
                if MP[i][k] < MP[j][k]:
                    MV[i][k] += 1
                if MP[i][k] > MP[j][k]:
                    MV[i][k] -= 1
    for i in range(len(MP)):
        for k in MP[i]:
            MP[i][k] += MV[i][k]

In [ ]:
SEEN = {k: {} for k in MP[0]}
CNT = {k: 0 for k in MP[0]}

t = 0
p2 = 1
periods_seen = 0

while periods_seen < 3:
    tick()
    KEY = {k: [] for k in MP[0]}
    for i in range(len(MP)):
        for k in MP[i]:
            KEY[k].append(MP[i][k])
            KEY[k].append(MV[i][k])
    KEY = {k: tuple(v) for k,v in KEY.items()}
    for k in KEY:
        if KEY[k] in SEEN[k]:
            if CNT[k] == 0:
                assert SEEN[k][KEY[k]] == 0
                # Say X repeats after tX, Y repeats after tY
                # Then (X,Y) will repeat after lcm(tX,tY) = tX*tY/gcd(tX,tY)
                # lcm(A,B,C) = lcm(lcm(A,B),C)
                p2 = p2 * t // math.gcd(p2, t)
                periods_seen += 1
                if periods_seen == 3:
                    print(p2)
                    break
            CNT[k] += 1
        SEEN[k][KEY[k]] = t
    t += 1

fin